### 꼭지점 추출 

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def extract_heart_corners(image_path: str):
    # 1. 이미지 불러오기
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # 2. 이미지 이진화 (Threshold 적용)
    _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
    
    # 3. 외곽선 검출 (Canny 엣지 검출기 사용)
    edges = cv2.Canny(thresh, 100, 200)
    
    # 4. 윤곽선 찾기 (findContours를 사용해 외곽선 점들을 찾음)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 5. 가장 큰 윤곽선을 선택
    max_contour = max(contours, key=cv2.contourArea)
    
    # 6. 윤곽선을 단순화하여 꼭짓점 추출
    epsilon = 0.01 * cv2.arcLength(max_contour, True)
    approx = cv2.approxPolyDP(max_contour, epsilon, True)
    
    # 7. 추출된 꼭짓점 좌표 리스트
    corner_points = [tuple(pt[0]) for pt in approx]
    
    # 8. 꼭짓점 상대 좌표 계산
    min_x = min([p[0] for p in corner_points])
    min_y = min([p[1] for p in corner_points])
    
    relative_points = [(x - min_x, y - min_y) for x, y in corner_points]
    
    return corner_points, relative_points

# 이미지 파일 경로 설정 (여기서 하트 이미지를 사용)
image_path = 'image.png'

# 꼭짓점 추출
corners, relative_corners = extract_heart_corners(image_path)

# 결과 출력 (여기서 relative_corners를 사용할 것)
print("상대 좌표:", relative_corners)


[ WARN:0@0.241] global loadsave.cpp:241 findDecoder imread_('image.png'): can't open/read file: check file path/integrity


ValueError: max() arg is an empty sequence

In [2]:
import folium

# 하트의 상대 좌표를 입력받아 지도 위에 그리기 (시작 지점으로 돌아가는 기능 포함)
def draw_heart_on_map(center_lat, center_lon, relative_corners, scale_factor=0.00005):
    # 지도 생성
    M = folium.Map(location=[center_lat, center_lon], zoom_start=17)

    # 하트 모양의 꼭짓점 계산 (상대 좌표 기반) - 크기를 키우기 위해 scale_factor 사용
    heart_coordinates = [(center_lat + rel_lat * scale_factor, center_lon + rel_lon * scale_factor) for rel_lat, rel_lon in relative_corners]

    # 시작 지점으로 다시 돌아가도록 좌표를 추가 (닫힌 도형)
    heart_coordinates.append(heart_coordinates[0])  # 첫 번째 좌표를 다시 추가

    # 하트 모양을 빨간 선으로 연결
    folium.PolyLine(locations=heart_coordinates, color='red').add_to(M)

    # 지도 저장
    M.save("heart_on_map_closed.html")

# 임의의 기준 좌표 (예시)
center_lat = 37.484208
center_lon = 126.929676

# 추출한 하트 이미지의 상대 좌표


# 하트의 크기를 키우기 위한 scale_factor 설정
scale_factor = 0.0001  # 이 값을 더 키우면 하트 크기가 커집니다.

# 지도 위에 하트 그리기
draw_heart_on_map(center_lat, center_lon, relative_corners, scale_factor)


NameError: name 'relative_corners' is not defined